In [1]:
import tensorflow as tf

(x_tr, y_tr), (x_ts, y_ts) = tf.keras.datasets.cifar10.load_data()
x_tr = x_tr / 255
x_ts = x_ts / 255

model = tf.keras.applications.MobileNet(input_shape = (32, 32, 3), 
                                        include_top = False, weights='imagenet')  
                                        # output은 빼겠음. include_top = false
model.trainable = False  # 훈련하지 않겠음

17225924/17225924 [==============================] - 1s 0us/step


In [3]:
t_model = tf.keras.models.Sequential([
    model, # MobileNet  (전이학습)
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

t_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 1, 1, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               131200    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 3,361,354
Trainable params: 132,490
Non-trainable params: 3,228,864
_________________________________________________________________


In [4]:
t_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
        # 원핫인코딩 안하면 sparse
        
# transfer learning
t_model.fit(x_tr, y_tr, epochs = 5, batch_size = 32, validation_data = (x_ts, y_ts))


c:\Python\Python39\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/5
1563/1563 [==============================] - 35s 14ms/step - loss: 2.1726 - accuracy: 0.2092 - val_loss: 2.1372 - val_accuracy: 0.2245
Epoch 2/5
1563/1563 [==============================] - 21s 14ms/step - loss: 2.1365 - accuracy: 0.2254 - val_loss: 2.1278 - val_accuracy: 0.2270
Epoch 3/5
1563/1563 [==============================] - 21s 13ms/step - loss: 2.1259 - accuracy: 0.2304 - val_loss: 2.1218 - val_accuracy: 0.2302
Epoch 4/5
1563/1563 [==============================] - 21s 13ms/step - loss: 2.1195 - accuracy: 0.2324 - val_loss: 2.1220 - val_accuracy: 0.2330
Epoch 5/5
1563/1563 [==============================] - 20s 13ms/step - loss: 2.1156 - accuracy: 0.2342 - val_loss: 2.1159 - val_accuracy: 0.2348
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 1, 1, 1024)       3228864   
 nal)                                                    

In [7]:
from keras import backend as K
import gc
K.clear_session()
gc.collect()

1113

In [11]:
# Fine-Tuning

t_model.trainable = True
t_model.summary()
t_model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),
                loss='sparse_categorical_crossentropy',
                metrics='accuracy')
t_model.fit(x_tr, y_tr, epochs=5, batch_size=32, validation_data=(x_ts, y_ts))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 1, 1, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 128)               131200    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 3,361,354
Trainable params: 3,339,466
Non-trainable params: 21,888
_________________________________________________________________


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.